The lab material presented here is based on the data provided by the [OpenFlights](https://openflights.org/data.html) dataset that contains information on airports, airlines and routes.

The [OpenFlights](https://openflights.org/data.html) dataset contains the ''Airport database'' that provides information for over 10,000 airports, train stations and ferry terminals spanning the globe. Each entry contains the following information:
* **Airport ID** - Unique OpenFlights identifier for this airport.
* **Name** - Name of airport. May or may not contain the City name.
* **City** - Main city served by airport. May be spelled differently from Name.
* **Country** - Country or territory where airport is located. See countries.dat to cross-reference to ISO 3166-1 codes.
* **IATA** - 3-letter IATA code. Null if not assigned/unknown.
* **ICAO** - 4-letter ICAO code. Null if not assigned.
* **Latitude** - Decimal degrees, usually to six significant digits. Negative is South, positive is North.
* **Longitude** - Decimal degrees, usually to six significant digits. Negative is West, positive is East.
* **Altitude** - In feet.
* **Timezone** - Hours offset from UTC. Fractional hours are expressed as decimals, eg. India is 5.5.
* **DST** - Daylight savings time. One of E (Europe), A (US/Canada), S (South America), O (Australia), Z (New Zealand), N (None) or U (Unknown). See also: Help: Time
* **Tz database time zone** - Timezone in "tz" (Olson) format, eg. "America/Los_Angeles".
* **Type** - Type of the airport. Value "airport" for air terminals, "station" for train stations, "port" for ferry terminals and "unknown" if not known. In airports.csv, only type=airport is included.
* **Source** - Source of this data. "OurAirports" for data sourced from OurAirports, "Legacy" for old data not matched to OurAirports (mostly DAFIF), "User" for unverified user contributions. In airports.csv, only source=OurAirports is included.

The data is UTF-8 (Unicode) encoded.

The [OpenFlights](https://openflights.org/data.html) dataset also contains the ''Route database'' which provides information for 59036 routes between 3209 airports on 531 airlines spanning the globe. Each entry contains the following information:
* **Airline** - 2-letter (IATA) or 3-letter (ICAO) code of the airline.
* **Airline ID** - Unique OpenFlights identifier for airline (see Airline).
* **Source airport** - 3-letter (IATA) or 4-letter (ICAO) code of the source airport.
* **Source airport ID** - Unique OpenFlights identifier for source airport (see Airport)
* **Destination airport** - 3-letter (IATA) or 4-letter (ICAO) code of the destination airport.
* **Destination airport ID** - Unique OpenFlights identifier for destination airport (see Airport)
* **Codeshare** - "Y" if this flight is a codeshare (that is, not operated by Airline, but another carrier), empty otherwise.
* **Stops** - Number of stops on this flight ("0" for direct)
* **Equipment** - 3-letter codes for plane type(s) generally used on this flight, separated by spaces

The data is ISO 8859-1 (Latin-1) encoded. The special value \N is used for "NULL" to indicate that no value is available, and is understood automatically by MySQL if imported.

## Airport dataset

We start by retrieving the dataset directly from the web using [urllib](https://docs.python.org/3/library/urllib.html) method from the standard python library.

In [1]:
import urllib.request
url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
u = urllib.request.urlopen(url)
rawdata = u.read()

We store the information retrieved to a local file so that we can work with the data without the need to re-download again and again. 

In [2]:
localFile = open("airports.dat", "wb")
localFile.write(rawdata)
localFile.close()

The files retrieved follows a comma-separated format, so once again we use the [CSVREADER](https://docs.python.org/2/library/csv.html) a standard python package as explained in [Reading and writing comma-separated data](http://opentechschool.github.io/python-data-intro/core/csv.html). 

Notice that since the file is using [UTF-8 encoding](https://en.wikipedia.org/wiki/UTF-8) we need to specify this when using the _open_ method.

In [3]:
import csv 

errors = 0
f = open("airports.dat", encoding='utf8')
airports = []
for row in csv.reader(f, delimiter=','):
    thisLine = []
    try:
        thisLine.append(int(row[0])) # Airport ID
        thisLine.append(row[1]) # Airport Name
        thisLine.append(row[2]) # City
        thisLine.append(row[3]) # Country
        thisLine.append(row[4]) # IATA/FAA Code
        thisLine.append(row[5]) # ICAO Code
        thisLine.append(float(row[6])) # Latitude
        thisLine.append(float(row[7])) # Longitude
        thisLine.append(float(row[8])) # Altitude
        thisLine.append(float(row[9])) # Timezone offset from UTC
        thisLine.append(row[10]) # Daylight savings code
        thisLine.append(row[11]) # Timezone
        thisLine.append(row[12]) # Type
        thisLine.append(row[13]) # Source

    except :
        errors += 1
        
    else:
        airports.append(thisLine)
        
print("Loaded", len(airports), "airports. Encountered", errors, "errors")

Loaded 6874 airports. Encountered 310 errors


## Routes dataset

We will now proceed by downloading and parsing the routes dataset.

In [4]:
url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'
u = urllib.request.urlopen(url)
rawdata = u.read()
localFile = open("routes.dat", "wb")
localFile.write(rawdata)
localFile.close()

In [9]:
errors = 0
f = open("routes.dat")
routes = []
for row in csv.reader(f):
    thisLine = []
    try:
        thisLine.append(row[0]) # IATA code
        thisLine.append(int(row[1])) # Airline ID
        thisLine.append(int(row[3])) # Source Airport ID
        thisLine.append(int(row[5])) # Destination Airport ID
        thisLine.append(int(row[7])) # Number of stops
        thisLine.append(row[8]) # Plane type        
        
    except :
        errors += 1
        
    else:
        routes.append(thisLine)
        
print("Loaded", len(routes), "routes. Encountered", errors, "errors")

Loaded 66765 routes. Encountered 898 errors


## Data as a Graph  

We will use the [networkX](http://networkx.github.io/) library to represent the dataset as a graph. We will represent each airport with a node in the graph. For each route connecting two airports, we will use an edge connecting the corresponding nodes.

In [7]:
import networkx as nx
G = nx.DiGraph()

### Inserting Nodes

We will use the ID of the airport as the main identifier of the node in the Directed Graph.

[networkX](http://networkx.github.io/) allows us to attach labels to each node and edge of the Graph. We will use this feature to store the information provided by the dataset.

In [8]:
for row in airports:
    G.add_node(row[0], id=row[0], 
               name=row[1], city=row[2], country=row[3],
               iata=row[4], icao=row[5],
               lat=row[6], long=row[7], alt=row[8],
               utc_offset=row[9], daylight=row[10], timezone=row[11],
               type=row[12], source=row[13])

### Inserting Edges

In a similar way we insert the edges in the Directed Graph.

Due to some errors occured while loading the airport dataset, some airports are missing. When we use the _add__edge_ method, if the nodes do not exist, they will be created at that point. These automatically created nodes will not have any labels attached. This may create some problems later on. So we will exclude all routes that involve an airport that is not loaded.

In [11]:
f = open("routes.dat")
for row in routes:    
    if row[2] in G.nodes() and row[3] in G.nodes():
        G.add_edge(row[2], row[3], 
                   airline=row[0], airlineID=row[1],
                  stops=row[4], aircraft=[5])

### Retrieving Nodes and Edges

Retrieving a node is done very easily based on the hashable value provided (in our case, the ID of the airport)

In [13]:
G.node[1555]

{'alt': 13.0,
 'city': 'Rome',
 'country': 'Italy',
 'daylight': 'E',
 'iata': 'FCO',
 'icao': 'LIRF',
 'id': 1555,
 'lat': 41.8002778,
 'long': 12.2388889,
 'name': 'Leonardo da Vinci–Fiumicino Airport',
 'source': 'OurAirports',
 'timezone': 'Europe/Rome',
 'type': 'airport',
 'utc_offset': 1.0}

Similarly we can retrieve an edge by providing the ID of the source and destination node.

In [14]:
G.edge[1555][1550]

{'aircraft': [5], 'airline': 'AZ', 'airlineID': 596, 'stops': 0}

In [15]:
G.edge[1550][1555]

{'aircraft': [5], 'airline': 'AZ', 'airlineID': 596, 'stops': 0}

### Graph Properties

[networkX](http://networkx.github.io/) provides algorithms for computing various metrics of the graph structure.

In [12]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 6874\nNumber of edges: 36116\nAverage in degree:   5.2540\nAverage out degree:   5.2540'

In [19]:
nx.average_node_connectivity(G)

KeyboardInterrupt: 